# 1️⃣ Training an Adapter for a Transformer model

In this notebook, we train an adapter for a **RoBERTa** ([Liu et al., 2019](https://arxiv.org/pdf/1907.11692.pdf)) model for sequence classification on a **sentiment analysis** task using [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers), the _AdapterHub_ adaptation of HuggingFace's _transformers_ library.

If you're unfamiliar with the theoretical parts of adapters or the AdapterHub framework, check out our [introductory blog post](https://adapterhub.ml/blog/2020/11/adapting-transformers-with-adapterhub/) first.

We train a **Task Adapter** for a pre-trained model here. Most of the code is identical to a full finetuning setup using HuggingFace's transformers. For comparison, have a look at the [same guide using full finetuning](https://colab.research.google.com/drive/1brXJg5Mokm8h3shxqPRnoIsRwHQoncus?usp=sharing).

For training, we use the [movie review dataset by Pang and Lee (2005)](http://www.cs.cornell.edu/people/pabo/movie-review-data/). It contains movie reviews  from Rotten Tomatoes which are either classified as positive or negative. We download the dataset via HuggingFace's [datasets](https://github.com/huggingface/datasets) library.

## Installation

First, let's install the required libraries:

In [1]:
!pip install -U adapter-transformers
!pip install datasets

## Dataset Preprocessing

Before we start to train our adapter, we first prepare the training data. Our training dataset can be loaded via HuggingFace `datasets` using one line of code:

In [2]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("yxchar/citation_intent-tlm")
dataset.num_rows

Using custom data configuration yxchar___citation_intent-tlm-4fc1a589669b5c4e
Reusing dataset csv (C:\Users\Junfe\.cache\huggingface\datasets\csv\yxchar___citation_intent-tlm-4fc1a589669b5c4e\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/3 [00:00<?, ?it/s]

{'train': 1688, 'test': 139, 'validation': 114}

Every dataset sample has an input text and a binary label:

In [3]:
dataset['train']['label'][0]

0

Now, we need to encode all dataset samples to valid inputs for our Transformer model. Since we want to train on `roberta-base`, we load the corresponding `RobertaTokenizer`. Using `dataset.map()`, we can pass the full dataset through the tokenizer in batches:

In [4]:
import numpy as np
nLabels = len(np.unique(dataset['train']['label']))
nLabels

6

In [5]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=512, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\csv\yxchar___citation_intent-tlm-4fc1a589669b5c4e\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-0c4be1345ad7d006.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

C:\Users\Junfe\AppData\Local\Temp/ipykernel_7696/1657904570.py:12: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  dataset.rename_column_("label", "labels")


Now we're ready to train our model...

## Training

We use a pre-trained RoBERTa model from HuggingFace. We use `RobertaModelWithHeads`, a class unique to `adapter-transformers`, which allows us to add and configure prediction heads in a flexibler way.

In [6]:
dataset['train']['labels']

tensor([0, 1, 0,  ..., 0, 2, 1])

In [7]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=nLabels,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

**Here comes the important part!**

We add a new adapter to our model by calling `add_adapter()`. We pass a name (`"rotten_tomatoes"`) and [the type of adapter](https://docs.adapterhub.ml/adapters.html#adapter-types) (task adapter). Next, we add a binary classification head. It's convenient to give the prediction head the same name as the adapter. This allows us to activate both together in the next step. The `train_adapter()` method does two things:

1. It freezes all weights of the pre-trained model so only the adapter weights are updated during training.
2. It activates the adapter and the prediction head such that both are used in every forward pass.

In [8]:
adaptername = 'CitationIntent'

# Add a new adapter
model.add_adapter(adaptername)
# Add a matching classification head
model.add_classification_head(
    adaptername,
    num_labels=nLabels
    # id2label={ 0: "👎", 1: "👍"}
  )
# Activate the adapter
model.train_adapter(adaptername)

For training, we make use of the `Trainer` class built-in into `transformers`. We configure the training process using a `TrainingArguments` object and define a method that will calculate the evaluation accuracy in the end. We pass both, together with the training and validation split of our dataset, to the trainer instance.

**Note the differences in hyperparameters compared to full finetuning.** Adapter training usually required a few more training epochs than full finetuning.

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 1688
    })
    test: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 139
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 114
    })
})

In [10]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=30,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

Start the training 🚀

In [11]:
trainer.train()

***** Running training *****
  Num examples = 1688
  Num Epochs = 30
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 4230


Step,Training Loss
200,1.305700
400,1.026500
600,0.860000
800,0.732600
1000,0.622700
1200,0.542200
1400,0.480300
1600,0.425700
1800,0.372100
2000,0.341200


Saving model checkpoint to ./training_output\checkpoint-500
Configuration saved in ./training_output\checkpoint-500\CitationIntent\adapter_config.json
Module weights saved in ./training_output\checkpoint-500\CitationIntent\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-500\CitationIntent\head_config.json
Module weights saved in ./training_output\checkpoint-500\CitationIntent\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\CitationIntent\head_config.json
Module weights saved in ./training_output\checkpoint-500\CitationIntent\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\CitationIntent\head_config.json
Module weights saved in ./training_output\checkpoint-500\CitationIntent\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-1000
Configuration saved in ./training_output\checkpoint-1000\CitationIntent\adapter_config.json
Module weights saved in ./training_output\checkpoint-10

TrainOutput(global_step=4230, training_loss=0.41491470624369087, metrics={'train_runtime': 871.0149, 'train_samples_per_second': 58.139, 'train_steps_per_second': 4.856, 'total_flos': 1.355545666142208e+16, 'train_loss': 0.41491470624369087, 'epoch': 30.0})

Looks good! Let's evaluate our adapter on the validation split of the dataset to see how well it learned:

In [12]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 114
  Batch size = 12


{'eval_loss': 1.1392338275909424,
 'eval_acc': 0.7719298245614035,
 'eval_runtime': 0.8902,
 'eval_samples_per_second': 128.061,
 'eval_steps_per_second': 11.233,
 'epoch': 30.0}

We can put our trained model into a `transformers` pipeline to be able to make new predictions conveniently:

At last, we can also extract the adapter from our model and separately save it for later reuse. Note the size difference compared to a full model!

In [13]:
import torch
device='cuda'

test_dataset = dataset['test']
bsz = 16
# bsz = test_dataset.num_rows
i = 0
batches = []
while i<test_dataset.num_rows:
  batches.append(test_dataset[i:i+bsz])
  i+=bsz


# Put model in evaluation mode
model.to(device)
model.eval()


# Tracking variables for storing ground truth and predictions 
predictions , true_labels = [], []

# Prediction Loop
for i,batch in enumerate(batches):
  print(i)

 
 
  # Unpack the inputs from our dataloader and move to GPU/accelerator 
 
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  labels = batch['labels'].to(device)

  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(input_ids, attention_mask=attention_mask, 
                         labels=labels)

  logits = outputs[1]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

0
1
2
3
4
5
6
7
8


In [14]:
labels = np.concatenate(true_labels)
prds = np.concatenate(predictions).argmax(axis=1)
from sklearn.metrics import f1_score, classification_report
f1_score(labels, prds, average='micro')

0.7769784172661871

In [15]:
print(classification_report(labels, prds, labels=list(range(nLabels)), digits=4))

              precision    recall  f1-score   support

           0     0.8182    0.8873    0.8514        71
           1     0.7037    0.7308    0.7170        26
           2     0.8947    0.6800    0.7727        25
           3     0.7500    0.6000    0.6667         5
           4     0.3750    0.4286    0.4000         7
           5     0.7500    0.6000    0.6667         5

    accuracy                         0.7770       139
   macro avg     0.7153    0.6544    0.6791       139
weighted avg     0.7833    0.7770    0.7761       139



In [16]:
adaptername = 'CitationIntent'

In [17]:
model.save_adapter("./final_adapter", adaptername)
!ls -lh final_adapter

Configuration saved in ./final_adapter\adapter_config.json
Module weights saved in ./final_adapter\pytorch_adapter.bin
Configuration saved in ./final_adapter\head_config.json
Module weights saved in ./final_adapter\pytorch_model_head.bin
'ls' is not recognized as an internal or external command,
operable program or batch file.


**Share your work!**

The next step after training is to share our adapter with the world via _AdapterHub_. [Read our guide](https://docs.adapterhub.ml/contributing.html) on how to prepare the adapter module we just saved and contribute it to the Hub!

➡️ Also continue with [the next Colab notebook](https://colab.research.google.com/github/Adapter-Hub/adapter-transformers/blob/master/notebooks/02_Adapter_Inference.ipynb) to learn how to use adapters from the Hub.